In [2]:
import carla
import random 
import time 
import numpy as np
import cv2
import math

client = carla.Client('localhost', 2000)
world = client.get_world()

bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

RuntimeError: time-out of 5000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000

In [11]:
# spawn vehicle
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

In [12]:
# set the spectator
spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [13]:
# set the camera
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')
camera_init_trans = carla.Transform(carla.Location(x=0.4, z=1.0))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

In [14]:
# def callback
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [15]:
# Get the camera dimensions and initialize the dictionary
image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()
camera_data = {'image': np.zeros(shape=(image_h, image_w, 4))}

In [16]:
# Start camera recording
camera.listen(lambda image: camera_callback(image, camera_data))

In [17]:
# set the control
vehicle.set_autopilot(False)

In [18]:
# OpenCV named window for rendering
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
cv2.waitKey(1)

# Game loop
while True:
    
    # Imshow renders sensor data to display
    cv2.imshow('RGB Camera', camera_data['image'])
    
    # Quit if user presses 'q'
    if cv2.waitKey(1) == ord('q'):
        break

# Close OpenCV window when finished
cv2.destroyAllWindows()

: 